In [ ]:
import requests
import tkinter as tk
from tkinter import messagebox
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Constants
API_KEY = "b3edece71a7dc08a010d449ff46075c1"
BASE_URL = "https://api.openweathermap.org/data/2.5/"
PAKISTAN_CITIES = [
    "Karachi", "Lahore", "Islamabad", "Quetta", "Peshawar",
    "Multan", "Faisalabad", "Rawalpindi", "Sialkot", "Hyderabad"
]

# Function to fetch weather data for a city
def fetch_weather():
    city = city_entry.get()
    if not city:
        messagebox.showerror("Error", "City name cannot be empty!")
        return

    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        display_weather(data)
    else:
        messagebox.showerror("Error", "City not found or Enter the correct city name(Karachi).")

# Function to display weather data
def display_weather(data):
    description = data['weather'][0]['description'].capitalize()
    temp = data['main']['temp']
    humidity = data['main']['humidity']
    pressure = data['main']['pressure']
    weather_label.config(
        text=f"Weather: {description}\nTemperature: {temp}°C\n"
             f"Humidity: {humidity}%\nPressure: {pressure} hPa"
    )

# Function to fetch and display the 10 coldest cities in Pakistan
def fetch_coldest_cities():
    temps = []
    for city in PAKISTAN_CITIES:
        url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            temps.append((city, data['main']['temp']))
    temps = sorted(temps, key=lambda x: x[1])[:10]
    cities, temperatures = zip(*temps)
    plot_bar_chart(cities, temperatures)

# Function to plot a bar chart with vertical city names
def plot_bar_chart(cities, temperatures):
    fig = Figure(figsize=(6, 4.5), dpi=70)  # Increased figure size
    ax = fig.add_subplot(111)
    ax.bar(cities, temperatures, color='skyblue')
    ax.set_title("10 Coldest Cities", fontsize=14)
    ax.set_ylabel("Temperature (°C)", fontsize=12)
    ax.set_xlabel("Cities", fontsize=12)

    # Make city names vertical and ensure they fit
    ax.set_xticks(range(len(cities)))
    ax.set_xticklabels(cities, rotation=45, fontsize=10, ha="right")

    # Adjust layout to prevent truncation
    fig.subplots_adjust(bottom=0.3)
    fig.tight_layout()

    # Embed plot in Tkinter GUI
    for widget in chart_frame.winfo_children():
        widget.destroy()

    canvas = FigureCanvasTkAgg(fig, master=chart_frame)
    canvas.get_tk_widget().pack()
    canvas.draw()

# GUI Setup
root = tk.Tk()
root.title("Weather App")
root.geometry("380x640")  # Mobile-like dimensions
root.configure(bg="#f7f7f7")  # Light background color

# Input field for city name
tk.Label(root, text="Enter City Name:", font=("Helvetica", 14), bg="#f7f7f7").pack(pady=10)
city_entry = tk.Entry(root, width=30, font=("Helvetica", 14))
city_entry.pack(pady=10)

# Buttons
btn_style = {
    "font": ("Helvetica", 14),
    "bg": "#4CAF50",
    "fg": "white",
    "bd": 0,
    "relief": "flat",
    "padx": 10,
    "pady": 10
}

tk.Button(root, text="Fetch Weather", command=fetch_weather, **btn_style).pack(pady=10)
tk.Button(root, text="Show Coldest Cities", command=fetch_coldest_cities, **btn_style).pack(pady=10)

# Display areas
weather_label = tk.Label(root, text="", font=("Helvetica", 12), bg="#f7f7f7", justify="center")
weather_label.pack(pady=20)

chart_frame = tk.Frame(root, bg="#f7f7f7")
chart_frame.pack(pady=20)

# Center align widgets
root.grid_columnconfigure(0, weight=1)

root.mainloop()
